### 比版本1 加入了对v_date长度的过滤

### 20181111批次数据____时间截断

In [1]:
# 日期 取值 集合  23,24,25 号日期异常

# d_list = ['180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
#           '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
#           '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
#           '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
#           '180711', '180712', '180713', '180714', '180715']

# d_list = ['180716', '180717', '180718', '180719', '180720', '180721', '180722', '180723', '180724', '180725', \
#           '180726', '180727', '180728', '180729', '180730', '180731', '180801', '180802', '180803', '180804', \
#           '180805', '180806', '180807', '180808', '180809', '180810', '180811', '180812', '180813', '180814',\
#           '180815', '180816', '180817', '180818', '180819', '180820', '180821', '180822', '180823', '180824',\
#           '180825', '180826']

d_list =[ '181105', '181106', '181107', '181108', '181109', '181110', '181111']
# d_list = ['181008', '181009', '181010', '181011', '181012', '181015', '181016', '181017', '181018', '181019']
# d_list = [ '180709', '180710','180711', '180712', '180713', '180714', '180715']

In [2]:
import pandas as pd
import time

# 替换一些命令前缀
cmd_class = ('perl','touch','sh','python','TDSQLLoader.pl','TDSQLLoaderHis.pl','TDDQCCheck.pl','TDDQCCheckHis.pl','TDDMCheck.pl')
def f_cmdline(cmd):
    if cmd in cmd_class:
        cmd = cmd.replace('TDSQLLoaderHis','TDSQLLoader')
        cmd = cmd.replace('TDDQCCheckHis','TDDQCCheck')
        return cmd
    elif cmd =='':
        return 'cmd_none'
    else:
        return 'cmd_other'
def f_s(s):
    return s[1:]
def f_d(s):
    return s[3:9]

### ajob表__处理

In [5]:
#分10份处理数据,10个日期   按v_date  划分；加入cmdline特征 
chunkSize = 100000
clums = ('order_id','task_type','job_name','job_id','odate','max_wait','confirm_flag','from_time','to_time','interval'\
,'next_time','priority','cpu_id','max_rerun','avg_runtime','status','state','delete_flag','start_time','end_time'\
,'rerun_counter','cmdline','cyclic','nodegroup','order_table','order_time','avg_start_time','cpu_time',\
 'prev_odate_rerun_counter','v_date')

cls_todo = [cl for cl in clums if cl not in ('order_id','v_date')]

for i in range(len(d_list)//10+1):
#     if i != 12 :
#         continue
    print(i)
    start = time.time()
    day_i = d_list[i*10:(i+1)*10]
    chunks = [[],[],[],[],[],[],[],[],[],[]]
    
    day_len = len(day_i)
    
    f = open(r'E:\aevt\20181111_AJOB_TMP_000AH000.DAT')
    reader = pd.read_csv(f, sep='|',header=None, iterator=True)
    
    loop = True
    i=0
    while loop:
        try:
            i += 1
            if i%100 == 0:
                print(i,end=' ')
    #             break
            chunk = reader.get_chunk(chunkSize)
            chunk = chunk[[0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,24,25,26,27,28,29,30,31]]
            chunk.columns = clums

            chunk['order_d'] = chunk['order_time'].apply(f_d)
            
            chunk['v_date_len'] = chunk['v_date'].apply(lambda x:len(x))
            chunk['server'] = chunk['v_date'].apply(lambda x:x[7:10])
            chunk['v_date'] = chunk['v_date'].apply(lambda x:x[1:7])
            
            chunk['cmdline'] = chunk['cmdline'].apply(lambda x: x.split(' ')[0])
            
            chunk = chunk[chunk['v_date_len']>10]
            del chunk['v_date_len']
            
            for k in range(day_len):
                chunks[k].append(chunk[chunk['order_d']==day_i[k]])

        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    
    

    
    for j in range(day_len):
        df = pd.concat(chunks[j], ignore_index=True)
        
        for cl in cls_todo:
            df[cl] = df[cl].apply(f_s)
            
        df['cmdline'] = df['cmdline'].apply(f_cmdline)
        #df['order_id'] = df[['order_id','server']].apply(lambda x:x[0]+x[1][1:],axis=1)
#         df['server'] = df['v_date'].apply(lambda x:x[7:11])
#         df['v_date'] = df['v_date'].apply(lambda x:x[:7])
        if df.shape[0]<1 :
            continue
        df.to_csv(r'E:\jupyter\CTM\data_src\ajob\ajob_%s_0_.txt'%(df['v_date'][0]),index=False)

    end = time.time()
    print('%0.2f S'%(end-start))

0
Iteration is stopped.
142.60 S


### 作业对应资源表处理

In [6]:
#  
#分10份处理数据,10个日期

chunkSize = 100000
clums = ('order_id','resource','quant','v_date')

cls_todo = clums[1:-1]

for i in range(len(d_list)//10+1):
#     if i != 12 :
#         continue
    print(i)
    start = time.time()
    day_i = d_list[i*10:(i+1)*10]
    chunks = [[],[],[],[],[],[],[],[],[],[]]
    
    day_len = len(day_i)
        
    f = open(r'E:\aevt\20181111_ALNKI_Q_TMP_000AH000.DAT')
    reader = pd.read_csv(f, sep='|',header=None, iterator=True)
    
    loop = True
    i=0
    while loop:
        try:
            i += 1
            if i%100 == 0:
                print(i,end=' ')
    #             break
            chunk = reader.get_chunk(chunkSize)
            chunk = chunk[[0,1,2,6]]
            chunk.columns = clums
            #chunk['v_date'] = chunk['v_date'].apply(f_s)  
            
            chunk['v_date_len'] = chunk['v_date'].apply(lambda x:len(x))
            chunk = chunk[chunk['v_date_len']>10]
            
            chunk['server'] = chunk['v_date'].apply(lambda x:x[7:10]) 
            chunk['v_date'] = chunk['v_date'].apply(lambda x:x[1:7]) 
            
            
            del chunk['v_date_len']
            for k in range(day_len):
                chunks[k].append(chunk[chunk['v_date']==day_i[k]])

        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    
    

    
    for j in range(day_len):
        df = pd.concat(chunks[j], ignore_index=True)
        
        for cl in cls_todo:
            df[cl] = df[cl].apply(f_s)
        df.to_csv(r'E:\jupyter\CTM\data_src\resorce\job_res_num_%s_.csv'%(df['v_date'][0]),index=False)

    end = time.time()
    print('%0.2f s'%(end-start))

0
100 Iteration is stopped.
48.45 s


### 依赖关系输入表__处理

In [7]:
#condition ——I 
#分10份处理数据,10个日期

chunkSize = 100000
clums = ('order_id','condition','odate','and_or','v_date')

cls_todo = clums[1:-1]

for i in range(len(d_list)//10+1):
#     if i != 12 :
#         continue
    print(i)
    start = time.time()
    day_i = d_list[i*10:(i+1)*10]
    chunks = [[],[],[],[],[],[],[],[],[],[]]
    
    day_len = len(day_i)
        
    f = open(r'E:\aevt\20181111_ALNKI_P_TMP_000AH000.DAT')
    reader = pd.read_csv(f, sep='|',header=None, iterator=True)
    
    loop = True
    i=0
    while loop:
        try:
            i += 1
            if i%100 == 0:
                print(i,end=' ')
    #             break
            chunk = reader.get_chunk(chunkSize)
            chunk = chunk[[0,1,2,3,6]]
            chunk.columns = clums
            #chunk['v_date'] = chunk['v_date'].apply(f_s)
            chunk['v_date_len'] = chunk['v_date'].apply(lambda x:len(x))
            chunk['server'] = chunk['v_date'].apply(lambda x:x[7:10])
            chunk['v_date_l'] = chunk['v_date']
            chunk['v_date'] = chunk['v_date'].apply(lambda x:x[1:7]) 
            
            chunk = chunk[chunk['v_date_len']>10]
            del chunk['v_date_len']
            for k in range(day_len):
                chunks[k].append(chunk[chunk['v_date']==day_i[k]])

        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    
    for j in range(day_len):
        df = pd.concat(chunks[j], ignore_index=True)
        
        for cl in cls_todo:
            df[cl] = df[cl].apply(f_s)
            
        df['up'] = df['condition'].apply(lambda x:x.split('-TO-')[0])
        df['down'] = df['condition'].apply(lambda x:x.split('-TO-')[-1])
        del df['condition']
        
        if df.shape[0]<1 :
            continue
        df.to_csv(r'E:\jupyter\CTM\data_src\condition_i/conditin_i_%s_.csv'%(df['v_date'][0]),index=False)

    end = time.time()
    print('%0.2f S'%(end-start))

0
Iteration is stopped.
40.90 S


### 依赖关系输出表__处理

In [8]:
#condition ——O
#分10份处理数据,10个日期
chunkSize = 100000
clums = ('order_id','condition','odate','sign','v_date')

cls_todo = clums[1:-1]

for i in range(len(d_list)//10+1):
#     if i != 12 :
#         continue
    print(i)
    start = time.time()
    day_i = d_list[i*10:(i+1)*10]
    chunks = [[],[],[],[],[],[],[],[],[],[]]
    
    day_len = len(day_i)
        
    f = open(r'E:\aevt\20181111_ALNKO_P_TMP_000AH000.DAT')
    reader = pd.read_csv(f, sep='|',header=None, iterator=True)
    
    loop = True
    i=0
    while loop:
        try:
            i += 1
            if i%100 == 0:
                print(i,end=' ')
    #             break
            chunk = reader.get_chunk(chunkSize)
            chunk = chunk[[0,1,2,3,5]]
            chunk.columns = clums
            #chunk['v_date'] = chunk['v_date'].apply(f_s）
            chunk['v_date_len'] = chunk['v_date'].apply(lambda x:len(x))
            chunk['server'] = chunk['v_date'].apply(lambda x:x[7:10]) 
            chunk['v_date'] = chunk['v_date'].apply(lambda x:x[1:7])
            
            chunk = chunk[chunk['v_date_len']>10]
            del chunk['v_date_len']
            for k in range(day_len):
                chunks[k].append(chunk[chunk['v_date']==day_i[k]])

        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    
    

    
    for j in range(day_len):
        df = pd.concat(chunks[j], ignore_index=True)
        
        for cl in cls_todo:
            df[cl] = df[cl].apply(f_s)
            
        df['up'] = df['condition'].apply(lambda x:x.split('-TO-')[0])
        df['down'] = df['condition'].apply(lambda x:x.split('-TO-')[-1])
        del df['condition']
        df.to_csv(r'E:\jupyter\CTM\data_src\condition_o\conditin_o_%s_.csv'%(df['v_date'][0]),index=False)

    end = time.time()
    print('%0.2f S'%(end-start))

0
100 Iteration is stopped.
84.08 S


### 事件表__处理

In [ ]:
##ALNKI_Q_TMP_  事件表
#分10份处理数据,10个日期

chunkSize = 100000
clums = ('evt_time','new_value','old_value','order_id','v_date')

cls_todo = clums[0:-1]

for i in range(len(d_list)//3+1):
#     if i != 12 :
#         continue
    print(i)
    start = time.time()
    day_i = d_list[i*3:(i+1)*3]
    chunks = [[],[],[],[],[],[],[],[],[],[]]
    
    day_len = len(day_i)
        
    f = open(r'E:\aevt\20181111_AEVT_TMP_000AH000.DAT')
    reader = pd.read_csv(f, sep='|',header=None, iterator=True)
    
    loop = True
    i=0
    while loop:
        try:
            i += 1
            if i%100 == 0:
                print(i,end=' ')
    #             break
            chunk = reader.get_chunk(chunkSize)
            chunk = chunk[[1,3,4,5,8]]
            chunk.columns = clums
            #chunk['v_date'] = chunk['v_date'].apply(f_s)
            chunk['v_date_len'] = chunk['v_date'].apply(lambda x:len(x))
            chunk = chunk[chunk['v_date_len']>10]
            
            chunk['server'] = chunk['v_date'].apply(lambda x:x[7:10]) 
            chunk['v_date'] = chunk['v_date'].apply(lambda x:x[1:7]) 
            chunk['evt_time'] = chunk['evt_time'].apply(lambda x:x[:-7])
            
            
            del chunk['v_date_len']
            for k in range(day_len):
                chunks[k].append(chunk[chunk['v_date']==day_i[k]])

        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    
    

    
    for j in range(day_len):
        df = pd.concat(chunks[j], ignore_index=True)
        
        for cl in cls_todo:
            df[cl] = df[cl].apply(f_s)
        
        df.to_csv(r'E:\jupyter\CTM\data_src\evet\evet_%s_.csv'%(df['v_date'][0]),index=False)

    end = time.time() 
    print('%0.2f mintue'%((end-start)/60))

0
100 200 300 400 500 600 Iteration is stopped.
3.49 mintue
1
100 200 300 400 500 600 Iteration is stopped.
3.25 mintue
2
100 200 300 400 500 

In [9]:
import pandas as pd
f = open(r'E:\jupyter\CTM\data_src\resorce\20181111_ARES_Q_TMP_000AH000.DAT')
res = pd.read_csv(f, sep='|',header=None)
res = res[[0,1,3,7]]
res.columns = ('resource','D','max_res','v_day')
res['D'] = res['D'].apply(lambda x:x[1:])
res['max_res'] = res['max_res'].apply(lambda x:x[1:])
res['server'] = res['v_day'].apply(lambda x:x[7:10])
res['v_day'] = res['v_day'].apply(lambda x:x[1:7])
print(res.head(10))

res.to_csv(r'E:\jupyter\CTM\data_src\resorce\res_1111_.csv',index=False)

                resource  D max_res   v_day server
0  QTY_DXP_SVR_WRITE_DGW  D       1  180602    001
1                 DXP101  D       0  180602    001
2     DXP_6750_OUT1_BAS3  D    1000  180602    001
3     DXP_6750_OUT2_CRD1  D    1000  180602    001
4                 SUB106  D    1000  180602    001
5                 SUB107  D    1000  180602    001
6     DXP_6750_OUT2_PFM1  D    1000  180602    001
7     DXP_6750_OUT2_DSP1  D    1000  180602    001
8                 RTL104  D    1000  180602    001
9     DXP_6750_OUT2_BAS1  D    1000  180602    001


In [4]:

import pandas as pd
f = open(r'E:\jupyter\CTM\data_src\resorce\20181111_ARES_Q_TMP_000AH000.DAT')
res = pd.read_csv(f, sep='|',header=None)
res = res[[0,1,3,7]]
res.columns = ('resource','D','max_res','v_day')
res['D'] = res['D'].apply(lambda x:x[1:])
res['max_res'] = res['max_res'].apply(lambda x:x[1:])
res['server'] = res['v_day'].apply(lambda x:x[7:10])

res['v_date_len'] = res['v_day'].apply(lambda x:len(x))
res = res[res['v_date_len']>10]
del res['v_date_len']
#res['v_day'] = res['v_day'].apply(lambda x:x[1:7])
print(res.head(10))

res.to_csv(r'E:\jupyter\CTM\data_src\resorce\res_1111_.csv',index=False)

                           resource  D max_res          v_day server
26960  QTY_GRP_SVR_PRCS_WEP@dssapp2  D      50  181105001_10    001
26961     QTY_DXP_SVR_PRCS_WEP@ds07  D     150  181105001_10    001
26962         QTY_DXP_SVR_WRITE_MI1  D       5  181105001_10    001
26963         QTY_DXP_DB_WRITE_CCF1  D       8  181105001_10    001
26964       QTY_DXP_SVR_WRITE_CCRMO  D       8  181105001_10    001
26965       QTY_DXP_SVR_READ_NQP_MT  D       5  181105001_10    001
26966          QTY_DXP_DB_READ_CIM1  D       8  181105001_10    001
26967         QTY_DXP_DB_WRITE_RC31  D       5  181105001_10    001
26968         QTY_DXP_DB_WRITE_BDS1  D       8  181105001_10    001
26969          QTY_DXP_DB_READ_BIM4  D       1  181105001_10    001


In [8]:
print(res[res['D']!='D'].head())

                      resource  D max_res   v_day server
257         QTY_TDDA_M_JOB_CNT  Q       0  180602    001
258         QTY_TDDA_M_JOB_CNT  Q       0  180602    001
259  QTY_DXP_SVR_PRCS_WEP@ds04  Q       0  180602    001
260         QTY_TDDA_M_JOB_CNT  Q       0  180602    001
261         QTY_TDDA_M_JOB_CNT  Q       0  180602    001
